In [65]:
import epitran, panphon
from wiktionaryparser import WiktionaryParser
import chinese_converter
import pandas as pd, csv
from functools import lru_cache

In [2]:
with open("../data/chinese/coordinate_compounds.txt") as f:
    lines = f.readlines()

In [3]:
coord_compounds = [line.split('.')[1][:2] for line in lines]

In [4]:
coord_compounds

['安危',
 '捭阖',
 '褒贬',
 '褒贬',
 '本末',
 '彼此',
 '标本',
 '表里',
 '宾主',
 '冰炭',
 '伯仲',
 '薄厚',
 '裁缝',
 '裁缝',
 '操纵',
 '产销',
 '长短',
 '唱和',
 '朝野',
 '臣民',
 '沉浮',
 '晨昏',
 '成败',
 '城镇',
 '乘除',
 '迟早',
 '出没',
 '出纳',
 '出入',
 '传习',
 '春秋',
 '雌雄',
 '粗细',
 '存亡',
 '大小',
 '旦夕',
 '得失',
 '弟妹',
 '弟兄',
 '颠末',
 '东西',
 '东西',
 '动静',
 '多寡',
 '多少',
 '多少',
 '恩怨',
 '儿女',
 '反正',
 '方圆',
 '肥瘦',
 '凤凰',
 '夫妇',
 '夫妻',
 '浮沉',
 '俯仰',
 '父母',
 '父子',
 '甘苦',
 '干支',
 '干群',
 '古今',
 '纲目',
 '高矮',
 '高低',
 '高下',
 '公婆',
 '公私',
 '功过',
 '功罪',
 '供求',
 '供需',
 '购销',
 '官兵',
 '广袤',
 '规矩',
 '贵贱',
 '寒热',
 '寒暑',
 '寒暄',
 '行列',
 '好歹',
 '好赖',
 '好恶',
 '黑白',
 '横竖',
 '横直',
 '厚薄',
 '呼吸',
 '呼应',
 '缓急',
 '毁誉',
 '晦明',
 '晦朔',
 '吉凶',
 '稼穑',
 '奖惩',
 '将士',
 '交接',
 '教学',
 '教学',
 '姐妹',
 '今昔',
 '进出',
 '进退',
 '经络',
 '举止',
 '巨细',
 '绝续',
 '军民',
 '开关',
 '考妣',
 '快慢',
 '宽窄',
 '来回',
 '来去',
 '来往',
 '来往',
 '劳逸',
 '劳资',
 '老少',
 '老小',
 '老幼',
 '冷暖',
 '离合',
 '利弊',
 '利钝',
 '利害',
 '利害',
 '廉耻',
 '买卖',
 '矛盾',
 '明灭',
 '男女',
 '南北',
 '内外',
 '浓淡',
 '平仄',
 '婆媳',
 '铺盖',
 '起伏',
 '起降',
 '起落',

0/262
安 has some problems
危 has some problems
捭 has some problems
阖 has some problems
褒 has some problems
贬 has some problems
本 has some problems
末 has some problems
5/262
彼 has some problems
此 has some problems
标 has some problems
表 has some problems
里 has some problems
宾 has some problems
主 has some problems
冰 has some problems


KeyboardInterrupt: 

In [5]:
from bs4 import BeautifulSoup

In [6]:
with open("/home/cuichenx/危 - Wiktionary.html") as f:
    text = f.read()
soup = BeautifulSoup(text, 'html.parser')

In [7]:
recon_table = soup.find(lambda tag: tag.name == 'tr' and "Reconstructions" in tag.text)
recons = {}
for mc_recon in recon_table.find_next_siblings('tr'):
    author, recon = mc_recon.text.lstrip().rstrip().split('\n\n')
    recons[author] = recon

## Download wiktionary entries and get MC pronunciation from there

In [8]:
import requests, os
session = requests.Session()
url = "https://en.wiktionary.org/wiki/{}?printable=yes"

def download_wiktionary(chinese_char):
    try:
        path = f'../data/chinese/wiktionary_entries/wiktionary_{chinese_char}.html'
        if not os.path.exists(path):
            response = session.get(url.format(chinese_char))
            content = response.text
            with open(path, 'w') as f:
                f.write(content)
            return content
        else:
            with open(path) as f:
                return f.read()
    except Exception as e:
        print("exception encountered \n", e)

    

In [75]:
def to_traditional(simp_char):
    manual_map = {
        '表里': '表裡',
        '唱和': '唱和',
        '沉浮': '沉浮',
        '出没': '出沒',
        '出纳': '出納',
        '春秋': '春秋',
        '粗细': '粗細',
        '浮沉': '浮沉',
        '俯仰': '俯仰',
        '干支': '干支',
        '干群': '幹群',
        '毁誉': '毀譽',
        '吉凶': '吉凶',
        '进出': '進出',
        '平仄': '平仄',
        '升降': '升降',
        '向背': '向背',
        '扬弃': '揚棄',
        '异同': '異同',
        '抑扬': '抑揚', 
        '舒卷': '舒捲', 
        '皂白': '皂白'
    }
    return manual_map.get(simp_char, chinese_converter.to_traditional(simp_char))

special_pronunciation = {
    '唱和': [1, 2],  # hè is pronunciation 3, but pronunciation 2 has no reconstruction
    '朝野': [2, 1],
    '行列': [2, 1],
    '好惡': [2, 2],
    '呼應': [1, 2],
    '將士': [2, 1],
    '教學': [2, 1],
    '老少': [1, 2],
    '乾坤': [2, 1],
    '勝敗': [2, 1],
    '睡覺': [1, 2],
    '消長': [1, 2],
    '臧否': [1, 2],
    '質量': [1, 2],
    '主從': [1, 2],
    '縱橫': [2, 1]
}

# for cc in coord_compounds:
#     print(cc, to_traditional(cc))
#     download_wiktionary(to_traditional(cc1))
#     download_wiktionary(to_traditional(cc2))

def flatten(t):
    return [item for sublist in t for item in sublist]

def get_pin1yin1(simp_char):
    special = {
        '捭阖': ['bai3', 'he2'],
    }
    return special.get(simp_char, flatten([cedict.hanzi[w][0] for w in cedict.tokenize(cc)]))



In [70]:
# https://stackoverflow.com/questions/8200349/convert-numbered-pinyin-to-pinyin-with-tone-marks
# I've got some Python 3 code that does this, and it's small enough to just put directly in the answer here.
import re
PinyinToneMark = {
    0: "aoeiuv\u00fc",
    1: "\u0101\u014d\u0113\u012b\u016b\u01d6\u01d6",
    2: "\u00e1\u00f3\u00e9\u00ed\u00fa\u01d8\u01d8",
    3: "\u01ce\u01d2\u011b\u01d0\u01d4\u01da\u01da",
    4: "\u00e0\u00f2\u00e8\u00ec\u00f9\u01dc\u01dc",
}
# 'nu:3' -> 'nǚ'
def decode_pinyin(s):
    s = s.lower()
    r = ""
    t = ""
    for c in s:
        if c >= 'a' and c <= 'z':
            t += c
        elif c == ':':
            assert t[-1] == 'u'
            t = t[:-1] + "\u00fc"
        else:
            if c >= '0' and c <= '5':
                tone = int(c) % 5
                if tone != 0:
                    m = re.search("[aoeiuv\u00fc]+", t)
                    if m is None:
                        t += c
                    elif len(m.group(0)) == 1:
                        t = t[:m.start(0)] + PinyinToneMark[tone][PinyinToneMark[0].index(m.group(0))] + t[m.end(0):]
                    else:
                        if 'a' in t:
                            t = t.replace("a", PinyinToneMark[tone][0])
                        elif 'o' in t:
                            t = t.replace("o", PinyinToneMark[tone][1])
                        elif 'e' in t:
                            t = t.replace("e", PinyinToneMark[tone][2])
                        elif t.endswith("ui"):
                            t = t.replace("i", PinyinToneMark[tone][3])
                        elif t.endswith("iu"):
                            t = t.replace("u", PinyinToneMark[tone][4])
                        else:
                            t += "!"
            r += t
            t = ""
    r += t
    return r

In [36]:
with open("../data/chinese/coordinate_compounds.txt") as f:
    lines = f.readlines()
coord_compounds = [line.split('.')[1][:2] for line in lines]
already_done = set([])
cedict = epitran.cedict.CEDictTrie('../../cedict/cedict_ts.u8')

In [101]:
already_done = set([])
MC_AUTHOR = 'LiRong'

pinyin = []
mc = []

for i, cc in enumerate(coord_compounds):
    if cc in already_done: continue
    already_done.add(cc)
    
    trad_cc = to_traditional(cc)
    pronunciation_n = special_pronunciation.get(trad_cc, [1, 1])
    pronun_mandarin = get_pin1yin1(cc)
    
    # add pinyin pronun
    pinyin.append(pronun_mandarin+[cc, ''])
    
    # add mc pronun
    this_mc = []
    for char, pn, mandarin in zip(trad_cc, pronunciation_n, pronun_mandarin):
        content = download_wiktionary(char)
        soup = BeautifulSoup(content, 'html.parser')
        recon_table = soup.find_all(lambda tag: tag.name == 'tr' and "Reconstructions" in tag.text)

        if len(recon_table) == 0:
            print(i, char, 'has no reconstructions. please look at it manually')
            this_mc.append('')
        # elif len(recon_table) > 1:
        #     print(char, 'is a 多音字, please look at it manually')
        #     stop='here'
        else:
            print(i, char)
            recon, mand_reflex = None, None
            for mc_recon in recon_table[pn-1].find_next_siblings('tr'):
                mc_recon_text = mc_recon.text.lstrip().rstrip()
                if mc_recon_text.startswith(MC_AUTHOR):
                    recon = mc_recon_text.split('\n\n')[1:]
                elif mc_recon_text.startswith('ExpectedMandarinReflex'):
                    mand_reflex = mc_recon_text.split('\n\n')[1:]
            if len(recon) == 1:
                to_append = recon[0]
            else:
                for rec, reflex in zip(recon, mand_reflex):
                    if reflex == decode_pinyin(mandarin):
                        to_append = rec
                        break
                else: # no break, no reflex matches, just use first recon
                    to_append = recon[0]
                    
            this_mc.append(to_append.replace('/', ''))

    mc.append(this_mc+[cc, ''])


0 安
0 危
1 捭
1 闔
2 褒
2 貶
4 本
4 末
5 彼
5 此
6 標
6 本
7 表
7 裡
8 賓 has no reconstructions. please look at it manually
8 主
9 冰
9 炭
10 伯
10 仲
11 薄
11 厚
12 裁
12 縫
14 操
14 縱
15 產
15 銷
16 長
16 短
17 唱
17 和
18 朝
18 野
19 臣
19 民
20 沉
20 浮
21 晨
21 昏
22 成
22 敗
23 城
23 鎮
24 乘
24 除
25 遲
25 早
26 出
26 沒
27 出
27 納
28 出
28 入
29 傳
29 習
30 春
30 秋
31 雌
31 雄
32 粗
32 細
33 存
33 亡
34 大
34 小
35 旦
35 夕
36 得
36 失
37 弟
37 妹
38 弟
38 兄
39 顛
39 末
40 東
40 西
42 動
42 靜
43 多
43 寡
44 多
44 少
46 恩
46 怨
47 兒
47 女
48 反
48 正
49 方
49 圓
50 肥
50 瘦
51 鳳
51 凰
52 夫
52 婦
53 夫
53 妻
54 浮
54 沉
55 俯
55 仰
56 父
56 母
57 父
57 子
58 甘
58 苦
59 干
59 支
60 幹
60 群
61 古
61 今
62 綱
62 目
63 高
63 矮
64 高
64 低
65 高
65 下
66 公
66 婆
67 公
67 私
68 功
68 過
69 功
69 罪
70 供
70 求
71 供
71 需
72 購
72 銷
73 官
73 兵
74 廣
74 袤
75 規
75 矩
76 貴
76 賤
77 寒
77 熱
78 寒
78 暑
79 寒
79 暄
80 行
80 列
81 好
81 歹
82 好
82 賴
83 好
83 惡
84 黑
84 白
85 橫
85 豎
86 橫
86 直
87 厚
87 薄
88 呼
88 吸
89 呼
89 應
90 緩
90 急
91 毀
91 譽
92 晦
92 明
93 晦
93 朔
94 吉
94 凶
95 稼
95 穡
96 獎
96 懲
97 將
97 士
98 交
98 接
99 教
99 學
101 姐
1

In [100]:
[(decode_pinyin(p[0]), decode_pinyin(p[1]), p[2]) for p in pinyin]

[]

In [108]:

pinyin_path = "../data/chinese/extracted_coordinate_compounds/pinyin.csv"
mc_path = "../data/chinese/extracted_coordinate_compounds/mc_lirong.csv"

pinyin = [[a.lower(), b.lower(), c, d] for (a, b, c, d) in pinyin]

pd.DataFrame(pinyin, columns=['cc1', 'cc2', 'rep', 'is_ABAC']).to_csv(pinyin_path, index=False, quoting=csv.QUOTE_ALL)
pd.DataFrame(mc, columns=['cc1', 'cc2', 'rep', 'is_ABAC']).to_csv(mc_path, index=False, quoting=csv.QUOTE_ALL)


In [110]:
mc

[['ʔɑn', 'ŋjue', '安危', ''],
 ['pɛX', 'ɣɑp̚', '捭阖', ''],
 ['pɑu', 'pjɛmX', '褒贬', ''],
 ['puənX', 'muɑt̚', '本末', ''],
 ['pjeX', 't͡sʰieX', '彼此', ''],
 ['piɛu', 'puənX', '标本', ''],
 ['piɛuX', 'liəX', '表里', ''],
 ['', 't͡ɕioX', '宾主', ''],
 ['piəŋ', 'tʰɑnH', '冰炭', ''],
 ['pɐk̚', 'ȡiuŋH', '伯仲', ''],
 ['buɑk̚', 'ɣuX', '薄厚', ''],
 ['d͡zᴀi', 'bioŋ', '裁缝', ''],
 ['t͡sʰɑu', 't͡sioŋH', '操纵', ''],
 ['ʃɛnX', 'siɛu', '产销', ''],
 ['ȡiaŋ', 'tuɑnX', '长短', ''],
 ['t͡ɕʰiaŋH', 'ɣuɑH', '唱和', ''],
 ['ȡjɛu', 'iaX', '朝野', ''],
 ['ʑiĕn', 'miĕn', '臣民', ''],
 ['ȡjəm', 'biu', '沉浮', ''],
 ['ʑiĕn', 'xuən', '晨昏', ''],
 ['ʑiɛŋ', 'baiH', '成败', ''],
 ['ʑiɛŋ', 'ȶjĕnH', '城镇', ''],
 ['d͡ʑiəŋ', 'ȡiɔ', '乘除', ''],
 ['ȡji', 't͡sɑuX', '迟早', ''],
 ['t͡ɕʰuiH', 'muət̚', '出没', ''],
 ['t͡ɕʰuiH', 'nᴀp̚', '出纳', ''],
 ['t͡ɕʰuiH', 'ȵiəp̚', '出入', ''],
 ['ȡjuɛn', 'ziəp̚', '传习', ''],
 ['t͡ɕʰiuĕn', 't͡sʰiu', '春秋', ''],
 ['t͡sʰie', 'ɣiuŋ', '雌雄', ''],
 ['d͡zoX', 'seiH', '粗细', ''],
 ['d͡zuən', 'miuaŋ', '存亡', ''],
 ['dɑiH', 'siɛuX', '大小', ''],


In [128]:
PINYIN_SYLLABLE = re.compile('''(?P<ons>[bpmfdtnlgkhjqxrwy]|[zcs]h?|)
                            (?P<rhy>[aeiou:]*n?g?|er)
                            (?P<ton>[12345])''', flags=re.X)

LIRONG_SYLLABLE = re.compile('''(?P<ons>t͡sʰ?|d͡z|t͡ɕʰ?|d͡ʑ|t͡ʃʰ?|d͡ʒ|[bmdnȡȵlszʃɕʑgɡŋʔxɣ]|[ptȶk]ʰ?|)
                            (?P<rhy>[ɑaᴀɛəeĕɐijoɔu]*[mnŋ]?)
                            (?P<ton>[HX]|[ptk]̚|)''', flags=re.X)

def verify_regex(regex, syl, verbose=False):
    m = regex.match(syl)
    if m is None:
        return False
    ret = syl == m.group('ons') + m.group('rhy') + m.group('ton')
    if not ret and verbose:
        print(syl, 'ons:', m.group('ons'), 'rhy:', m.group('rhy'), 'ton:', m.group('ton'), 'is bad')
    return ret

## verify pinyin regex
print(all([verify_regex(PINYIN_SYLLABLE, p[i], True) for i in range(2) for p in pinyin]))
## verify mc regex
print(all([verify_regex(LIRONG_SYLLABLE, p[i], True) for i in range(2) for p in mc]))


True
True
